In [5]:
import requests
import pandas as pd
import pickle
import numpy as np

In [6]:
filename = 'data/modelo_main.pkl'
with open(filename, 'rb') as file:
    loaded_model_data = pickle.load(file)

loaded_model = loaded_model_data['model']
X_train = loaded_model_data['X_train']
X_test = loaded_model_data['X_test']
y_train = loaded_model_data['y_train']
y_test = loaded_model_data['y_test']
data = loaded_model_data['data']

In [7]:
# El objetivo de este notebook es comprobar que todo este correctamente en la api.

In [8]:
data_original = pd.read_csv('data/reviews_data.csv').drop(columns=['reseñas'])
data_original.head(2)

,IA,longitud,longitud_promedio,palabras_unicas,signos_de_puntuación,frecuencia_pronombres,variedad_lexica,entropia_lexica,expresiones,tfidf,palabras_complejas,tecnicismos,polaridad,subjetividad,coherencia
0,0,297,4.066667,0.733333,6,0,0.733333,5.267482,0,5.640761e-02,3,0,0.0,0.0,0.92
1,0,99,5.375000,1.000000,2,0,1.000000,4.000000,0,5.551115e-17,2,0,0.0,0.0,1.00


In [9]:
# Intentemos acortar el proceso con funciones

def comparar_respuestas(endpoint:str, data_response: pd.DataFrame):
    api_rul = f'http://127.0.0.1:8000/reviews/{endpoint}'
    response = requests.get(api_rul).json()

    if type(response) == list:
        api_response = pd.DataFrame(response).reset_index().sort_index(axis=0).drop(columns=['index']).sort_index(axis=1)
        data_response = data_response.reset_index().sort_index(axis=0).drop(columns=['index']).sort_index(axis=1)

        are_equial = api_response.equals(data_response)
        print(f"Son los dataframe iguales -> {are_equial}")
        
    else:
        api_response = pd.DataFrame([response]).reset_index().sort_index(axis=0).drop(columns=['index']).sort_index(axis=1)
        data_response = data_response.reset_index().sort_index(axis=0).drop(columns=['index']).sort_index(axis=1)

        are_equial = api_response.equals(data_response)
        print(f"Son los dataframe iguales -> {are_equial}")

    return api_response.head()

In [10]:
# Todas las muestras
endpoint = 'all'
data_response = data_original

comparar_respuestas(endpoint=endpoint, data_response=data_response)

Son los dataframe iguales -> True


,IA,coherencia,entropia_lexica,expresiones,frecuencia_pronombres,longitud,longitud_promedio,palabras_complejas,palabras_unicas,polaridad,signos_de_puntuación,subjetividad,tecnicismos,tfidf,variedad_lexica
0,0,0.920000,5.267482,0,0,297,4.066667,3,0.733333,0.0,6,0.0,0,5.640761e-02,0.733333
1,0,1.000000,4.000000,0,0,99,5.375000,2,1.000000,0.0,2,0.0,0,5.551115e-17,1.000000
2,0,0.857143,5.710972,0,0,436,4.402439,5,0.695122,0.0,7,0.0,0,5.749596e-02,0.707317
3,0,0.823529,5.880116,0,1,657,4.124031,5,0.581395,0.0,10,0.0,0,7.866846e-02,0.604651
4,0,0.953488,5.953860,0,0,596,4.109244,4,0.638655,0.0,11,0.0,0,7.357145e-02,0.655462


In [11]:
# Los datos del train
endpoint = 'train'
data_response = pd.concat([X_train, y_train], axis=1)

comparar_respuestas(endpoint=endpoint, data_response=data_response)

Son los dataframe iguales -> True


,IA,coherencia,entropia_lexica,expresiones,frecuencia_pronombres,longitud,longitud_promedio,palabras_complejas,palabras_unicas,polaridad,signos_de_puntuación,subjetividad,tecnicismos,tfidf,variedad_lexica
0,1,0.952381,4.971928,0,0,248,5.350000,3,0.825000,0.5625,5,0.7,0,0.053853,0.825000
1,1,0.905983,6.729701,0,0,1415,4.775100,22,0.586345,1.0000,25,0.9,0,0.060800,0.606426
2,1,0.936000,6.890773,0,0,1408,4.888889,18,0.637860,0.0000,25,0.0,0,0.055777,0.658436
3,1,1.000000,4.657882,0,0,170,4.333333,3,0.818182,0.0000,5,0.0,0,0.057571,0.818182
4,0,1.000000,3.852169,0,0,75,3.470588,0,0.882353,0.0000,0,0.0,0,0.066617,0.882353


In [12]:
# Los datos del test
endpoint = 'test'
data_response = pd.concat([X_test, y_test], axis=1)

comparar_respuestas(endpoint=endpoint, data_response=data_response)

Son los dataframe iguales -> True


,IA,coherencia,entropia_lexica,expresiones,frecuencia_pronombres,longitud,longitud_promedio,palabras_complejas,palabras_unicas,polaridad,signos_de_puntuación,subjetividad,tecnicismos,tfidf,variedad_lexica
0,0,1.0,3.773557,0,0,59,3.000000,0,0.933333,0.000,0,0.0,0,7.136240e-02,0.933333
1,1,1.0,4.521641,0,1,164,5.035714,2,0.857143,0.000,4,0.0,0,5.649248e-02,0.857143
2,1,1.0,3.584963,0,0,82,6.083333,2,1.000000,0.000,2,0.0,0,0.000000e+00,1.000000
3,1,1.0,5.334963,0,0,265,4.625000,2,0.875000,0.625,4,0.6,0,3.846154e-02,0.875000
4,1,1.0,3.321928,0,0,60,5.200000,1,1.000000,0.000,1,0.0,0,5.551115e-17,1.000000


In [13]:
# Observación n del dataset, probamos con la 3
endpoint = 'observation/4'
data_response = data_original.iloc[[4]].astype('float')

comparar_respuestas(endpoint=endpoint, data_response=data_response)

Son los dataframe iguales -> True


,IA,coherencia,entropia_lexica,expresiones,frecuencia_pronombres,longitud,longitud_promedio,palabras_complejas,palabras_unicas,polaridad,signos_de_puntuación,subjetividad,tecnicismos,tfidf,variedad_lexica
0,0.0,0.953488,5.95386,0.0,0.0,596.0,4.109244,4.0,0.638655,0.0,11.0,0.0,0.0,0.073571,0.655462


In [14]:
# Hacemos una query de todo el dataset
endpoint = 'query?IA=1&longitud=297&subjetividad=0&tfidf=0.074756940702484'
valor_exacto = 0.074756940702484
mascara = np.isclose(data_original['tfidf'], valor_exacto)
data_response = data_original[(data_original['IA']==1) & (data_original['longitud']==297) & (data_original['subjetividad']==0) & mascara]

comparar_respuestas(endpoint=endpoint, data_response=data_response)

Son los dataframe iguales -> False


,IA,coherencia,entropia_lexica,expresiones,frecuencia_pronombres,longitud,longitud_promedio,palabras_complejas,palabras_unicas,polaridad,signos_de_puntuación,subjetividad,tecnicismos,tfidf,variedad_lexica
0,1,1.0,5.231468,0,0,297,4.666667,5,0.777778,0.0,8,0.0,0,0.074757,0.777778


In [15]:
# Por algun motivo da false, pero es exactamente la misma busqueda, comprobamos: (estamos imitando el código dentro de la función comprobar_respuestas)
valor_exacto = 0.074756940702484
mascara = np.isclose(data_original['tfidf'], valor_exacto)
data_response = data_original[(data_original['IA']==1) & (data_original['longitud']==297) & (data_original['subjetividad']==0) & mascara]
data_response.reset_index().sort_index(axis=0).drop(columns=['index']).sort_index(axis=1)

,IA,coherencia,entropia_lexica,expresiones,frecuencia_pronombres,longitud,longitud_promedio,palabras_complejas,palabras_unicas,polaridad,signos_de_puntuación,subjetividad,tecnicismos,tfidf,variedad_lexica
0,1,1.0,5.231468,0,0,297,4.666667,5,0.777778,0.0,8,0.0,0,0.074757,0.777778


In [16]:
# Predicciones
endpoint = 'predictions'
y_pred_test = loaded_model.predict(X_test).tolist()
y_pred_train = loaded_model.predict(X_train).tolist()
data_response = pd.DataFrame({
    'predicciones_test': [y_pred_test],
    'predicciones_train': [y_pred_train]
})

comparar_respuestas(endpoint=endpoint, data_response=data_response)

Son los dataframe iguales -> True


,predicciones_test,predicciones_train
0,"[0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, ...","[1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, ..."
